<a href="https://colab.research.google.com/github/anderfrago/trasladar-notas/blob/main/2021_CompartirCompetenciasClave.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#@markdown # TRASLADAR NOTAS AL ALUMNADO
#@markdown El siguente Notebook permite trasladar las líneas de un sheet al nombre de alumno establecido en la primera columna y compartirlo con el correo de la segunda columna.

#@markdown *   El listado de alumnos comienza a partir de la línea X.
#@markdown *   Primera columna nombre del alumno, la cabecera (X primeras líneas) y la línea actual se copia a un excel dentro de una carpeta con este nombre
#@markdown *   Segunda columna correo del alumno para compartir la carpeta.

#@markdown Las notas de las celdas de sheet aparecen como comentarios, el alumnado puede resolverlas y agregar comentarios. De esta manera se fomenta el proceso de retroalimentación

#@markdown ### Introduce nombre del archivo: (**obligatorio**)
#@markdown *Sin espacios en blanco*
nombre_excel_notas = "Copia CC 2Dam" #@param {type:"string"} 
#@markdown ### Introduce la ruta al directorio: (**obligatorio**)
#@markdown Introduce la **ubicación donde se encuentra el archivo** de notas para trasladar a los alumnos.
#@markdown *En caso de especificar la ruta completa el proceso será maś rápido*
ruta_directorio = "4Vientos/2021/Notak/pro b@k" #@param {type:"string"} 
path =  "/content/drive/MyDrive/"+ruta_directorio
filenombre_excel_notas = nombre_excel_notas+'.xlsx'

extension = "Google Sheet" #@param [ "Excel", "Google Sheet"] {type:"string"}

#@markdown ### Introduce nombre de la hoja/pestaña del documento EXCEL: (opcional)
#@markdown *En caso de no especificar se traslada la primera pestaña*
nombre_hoja = "NOTAS" #@param {type:"string"} 

#@markdown ## Cabecera: 
#@markdown ### Introduce el número de líneas de cabecera: (**obligatorio**)
numero_cabecera =  7#@param {type:"number"} 
#@markdown ### Introduce el nombre del archivo plantilla: (opcional)
#@markdown En caso de hacer uso de columnas colapsables es requisito crear una plantilla.
#@markdown *Consiste en crear una copia del EXCEL sin filas de alumnos.*
plantilla_cabecera =  ""#@param {type:"string"} 

#@markdown ## Columna nombre y correo: 
#@markdown ### Introduce la columna con los nombres de los alumnos
letra_columna_nombre =  "B"#@param {type:"string"} 
#@markdown ### Introduce la columna con los correos de los alumnos
letra_columna_correo =  "C"#@param {type:"string"} 

# Get positional value from Sheet column
def positionfromcolumn(l) :
  num_array = [ord(x.upper()) - 64 for x in l]
  result = 0
  for i, v in enumerate(num_array):
    result = i*26+v
  return result

col_nombre = positionfromcolumn(letra_columna_nombre)
#print(col_nombre)
col_correo = positionfromcolumn(letra_columna_correo)
#print(col_correo)

import requests
import os
from google.colab import drive as colabdrive
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Copy grades to excel
!pip install openpyxl > /dev/null
import openpyxl
import pandas as pd
import time
import shutil
from openpyxl import load_workbook
from copy import copy
import numpy as np


class TranslateGrades:

  def __init__(self):
      # Google Authentication
      self._gauth = None
      # Google Drive Python
      self._drive = None

  @property
  def drive(self):
      return self._drive

  @drive.setter
  def drive(self, value):
      self._drive = value

  @drive.deleter
  def drive(self):
      del self._drive
  @property
  def gauth(self):
      return self._gauth

  @gauth.setter
  def gauth(self, value):
      self._gauth = value

  @gauth.deleter
  def gauth(self):
      del self._gauth

  def connectgdrive(self): 
    # colabdrive.flush_and_unmount()
    colabdrive.mount('/content/drive', force_remount=True)

  def sdkaccess(self):  
    # Launch Google Authentication for first time
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    return drive, gauth

  def search(self, drive, gauth):    
    if extension == "Excel" :
      print("Búsqueda de Excel")
      listed = drive.ListFile({'q': " title contains '"+filenombre_excel_notas+"'"}).GetList()
    else :
      print("Búsqueda de GSheet")
      listed = drive.ListFile({'q': " mimeType ='application/vnd.google-apps.spreadsheet' and title contains '"+nombre_excel_notas+"'"}).GetList()

    #try: 
    # Last modified file
    file = listed[0]
    #print(file['id'])
    file_id = file['id']
    if file_id :
      print("Archivo de notas localizado exitosamente!")

    if extension == "Excel" :
      downloaded = drive.CreateFile({'id': file_id})
      downloaded.GetContentFile(filenombre_excel_notas)
    else :
      spreadsheetId = file_id # Please set the Spreadsheet ID.
      url = 'https://docs.google.com/spreadsheets/d/' + spreadsheetId + '/export?format=xlsx&id=' + spreadsheetId
      headers = {'Authorization': 'Bearer ' + gauth.credentials.access_token}
      res = requests.get(url, headers=headers)
      with open(filenombre_excel_notas, 'wb') as f:
          f.write(res.content)    
    #except:
    #  print("Error en la busqueda de  "+ filenombre_excel_notas)

  def copygrades(self):
    os.chdir('/content')
    # Work Book used to copy formats and comments
    wb_comments = openpyxl.load_workbook(filenombre_excel_notas)
    # BUG Calculate formulas values
    wb_comments.save(filename = 'tmp.xlsx')
    wb_comments.close()
    wb_obj = openpyxl.load_workbook('tmp.xlsx')
    
    # SELECT SHEET BY NAME FROM FORM
    if nombre_hoja :
      print("Selección por nombre de hoja", nombre_hoja)
      #ws_obj = wb_obj.worksheets[0]
      ws_comments = wb_comments[nombre_hoja]
      # Grade values are read from dataframe instead of Excel to resolve excel formulas
      df = pd.read_excel(open(filenombre_excel_notas,'rb'),header=None,sheet_name=nombre_hoja)
    else :
      # Select First Worksheet
      print("seleccionada primera pestaña")
      #ws_obj = wb_obj.worksheets[0]
      ws_comments = wb_comments.worksheets[0]
      # Grade values are read from dataframe instead of Excel to resolve excel formulas
      df = pd.read_excel(open(filenombre_excel_notas,'rb'),header=None)


    ## Process of transfering notes to students folder
    ###
    print("Comienza el proceso de trasladar las notas al alumnado")
    for index, row in df.iloc[numero_cabecera:].iterrows():

      # Get name to create the folder and contact mail
      name = row.values[col_nombre-1]  
      email = row.values[col_correo-1]
      # Creation of students folder
      print("\t - Trasladando notas al alumn@  "+str(name))
  #    try: 
      os.chdir(path)
      if not os.path.isdir(name) :
          print("\t\t Primera vez que se trasladan las notas, creación del directorio del alumn@")
          os.mkdir(name)
          time.sleep(1)

      os.chdir(name)
      # Transformr from numpy.ndarray to excel values
      row_list = row.values.tolist()

      file_name = name+"__"+filenombre_excel_notas


      #try :
      if plantilla_cabecera :
        print("Copiando la plantilla de cabecera")
        ## COPY TEMPLATE
        template=path+"/"+plantilla_cabecera+".xlsx"
        shutil.copyfile(template, file_name)
          
        time.sleep(1)
        # Load excel
        wb = openpyxl.load_workbook(file_name)
        ws = wb.worksheets[0]
      else :
        # Creation of excel
        wb = openpyxl.Workbook()
        wb.save(file_name)

        ws = wb.worksheets[0]
        ## COPY HEADER
        print("Generando cabecera")
        for index_header, row_header in df.iloc[:numero_cabecera].iterrows():
          row_list_header = row_header.values.tolist()
          for idx_header, val_header in enumerate(row_list_header):
            ws.cell(row=index_header+1, column=idx_header+1).value =val_header
            ws.cell(row=index_header+1, column=idx_header+1).comment = ws_comments.cell(row=index_header+1, column=idx_header+1).comment
            if ws_comments.cell(row=index_header+1, column=idx_header+1).has_style:
                  ws.cell(row=index_header+1, column=idx_header+1).font = copy(ws_comments.cell(row=index_header+1, column=idx_header+1).font)
                  ws.cell(row=index_header+1, column=idx_header+1).border = copy(ws_comments.cell(row=index_header+1, column=idx_header+1).border)
                  ws.cell(row=index_header+1, column=idx_header+1).fill = copy(ws_comments.cell(row=index_header+1, column=idx_header+1).fill)
                  ws.cell(row=index_header+1, column=idx_header+1).number_format = copy(ws_comments.cell(row=index_header+1, column=idx_header+1).number_format)
                  ws.cell(row=index_header+1, column=idx_header+1).protection =copy( ws_comments.cell(row=index_header+1, column=idx_header+1).protection)
                  ws.cell(row=index_header+1, column=idx_header+1).alignment = copy(ws_comments.cell(row=index_header+1, column=idx_header+1).alignment)

      ## ADDING STUDENTS VALUES 
      print(row_list)
      for idx, val in enumerate(row_list):
        ws.cell(row=numero_cabecera+1, column=idx+1).value = val
        ws.cell(row=numero_cabecera+1, column=idx+1).comment =ws_comments.cell(row=index+1, column=idx+1).comment
        if ws_comments.cell(row=numero_cabecera+1, column=idx+1).has_style:
          ws.cell(row=numero_cabecera+1, column=idx+1).font = copy(ws_comments.cell(row=index+1, column=idx+1).font)
          ws.cell(row=numero_cabecera+1, column=idx+1).border = copy(ws_comments.cell(row=index+1, column=idx+1).border)
          ws.cell(row=numero_cabecera+1, column=idx+1).fill = copy(ws_comments.cell(row=index+1, column=idx+1).fill)
          ws.cell(row=numero_cabecera+1, column=idx+1).number_format = copy(ws_comments.cell(row=index+1, column=idx+1).number_format)
          ws.cell(row=numero_cabecera+1, column=idx+1).protection = copy(ws_comments.cell(row=index+1, column=idx+1).protection)
          ws.cell(row=numero_cabecera+1, column=idx+1).alignment = copy(ws_comments.cell(row=index+1, column=idx+1).alignment)
    
    
      wb.save(file_name)
      wb.close()
      #  except:
      #    print("ERROR: Revisa el nombre del fichero plantilla: "+template)
      #    break
  #    except:
  #      print("ERROR: Revisa la ruta objetivo: "+path)
  #      break
    #return df

  def share(self, drive, gauth):
    ###
    ## Process of transfering notes to students folder
    ###
    os.chdir('/content')
    if nombre_hoja:
      df = pd.read_excel(open(filenombre_excel_notas,'rb'),header=None,sheet_name=nombre_hoja)
    else :
      df = pd.read_excel(open(filenombre_excel_notas,'rb'),header=None)
    
    for index, row in df.iloc[numero_cabecera:].iterrows():
      # Get name to create the folder and contact mail
      name = row.values[col_nombre-1]  
      email = row.values[col_correo-1].strip()
      print(name)
      # Creation of students folder
      os.chdir(path)
      #print(os.getcwd())
      #os.chdir(name)
      # Transform from numpy.ndarray to excel values
      row_list = row.values.tolist()
      # file_name = name+filenombre_excel_notas
      listed = drive.ListFile({'q': "title: '"+name+"'"}).GetList()
      # Get las modified version
      file = listed[0]
      print("Compartiendo notas con "+email)

      try:
        file.InsertPermission({'type': 'user','value': email,'role': 'writer'})
      except:
        print("An exception occurred with email",email)



##
# Application starting point
## 
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  app = TranslateGrades()
  print("Funcionalidades inicializadas...")
else:
  print('Not running on CoLab')


# VALIDAR CUENTA GOOGLE

In [ ]:

#@markdown Para usar la aplicación es necesario validar la cuenta de Google 2 veces.
#@markdown * La primera para instalar GDrive en la máquina en la nube.*
#@markdown * La segunda para usar las herramientas de desarrollo de GCloud.*

#try :
#  print("A continuación debes acceder a la URL que aparezca")
#  print("Despues, valida tu cuenta y pega el código de confirmación en la cajetilla")
app.connectgdrive()
#except:
#  print("ERROR: Conectando a Google Drive. ")

#try:
#  print("Repite el proceso de validación para dar acceso al SDK de Google Drive")
drive, gauth = app.sdkaccess()
#except:
#  print("ERROR: Obteniendo permisos de acceso al SDK de Google Drive")



# GENERAR NOTAS PARA COMPARTIR

In [ ]:
#@markdown ## Buscar fichero de notas en Google Drive
app.search(drive, gauth)

In [ ]:
#@markdown ## Copiar las notas al alumando
#@markdown En la carpeta se genera un excel y se copian las líneas de cabecera junto la nota del alumno
app.copygrades()

# ENVIAR NOTIFICACIÓN


In [ ]:
#@markdown ## Compartir las carpetas con los alumnos
#@markdown Se comparte la carpeta con nombre del alumn@ y se notificará al alumnado vía mail
app.share(drive, gauth)

# REINICIAR MÁQUINA



In [ ]:
#@markdown ## En caso de querer trasladar multiples documentos

#Kill previus machine
quit()